Importing necessary libraries


In [1]:
from Bio import Entrez
from Bio import SeqIO
from Bio.SeqUtils import nt_search
from Bio.Seq import Seq

# Function to download a FASTA file given an accession number
# def downloadFastaFile(accesion_num, output_file):
#     """
#     Downloads a nucleotide sequence in FASTA format from the NCBI nucleotide database
#     using the provided accession number and saves it to the specified output file.

#     Parameters:
#     - accesion_num (str): The accession number of the nucleotide sequence to be downloaded.
#     - output_file (str): The name of the file where the downloaded FASTA sequence will be saved.

#     Returns:
#     None
#     """
#     Entrez.email = "masoodmohd9250@gmail.com"
#     handle = Entrez.efetch(db="nucleotide", id=accesion_num, rettype="fasta", retmode="text")
#     record = handle.read()
#     handle.close()

#     with open(output_file, "w") as file:
#         file.write(record)


    


Download PDC gene 1,5,6 from Saccharomyces cerevisiae

In [2]:
Entrez.email = "masood22299@iiitd.ac.in"

def download_gene_sequence(accession_number, start, end, pdc_gene_number, reverse_complement=False):
    """
    Fetches the gene sequence from NCBI using accession number, start, and end positions.
    Optionally takes the reverse complement if specified.
    """
    print("PDC ", pdc_gene_number , "sequence download started")
    handle = Entrez.efetch(db="nucleotide",
                           id=accession_number,
                           rettype="fasta", 
                           seq_start=start, 
                           seq_stop=end,
                           expect = 0.01)
    seq_record = SeqIO.read(handle, "fasta")
    handle.close()

    if reverse_complement:
        seq_record.seq = seq_record.seq.reverse_complement()

    seq_file_pdc1 = "PDC"  + str(pdc_gene_number) + ".fasta"
    with open(seq_file_pdc1, "w") as seq_output_pdc1:
        SeqIO.write(seq_record, seq_output_pdc1, "fasta")
    print("PDC ", pdc_gene_number , "downloaded successfully")
    print("\n--------------------------------------------------\n")
    return seq_record

# Accession number  PDC1 gene sequence
accession_number_pdc1 = "NC_001144.5"

# Start end index of the  of the PDC1 gene sequence
start_pdc1 = 232390
end_pdc1 = 234081



# Downlaoding PDC1 gene with reverse complement

seq_record_pdc1 = download_gene_sequence(accession_number_pdc1, start_pdc1, end_pdc1, 1, reverse_complement=True)




# Accession number  PDC5 gene sequence
accession_number_pdc5 = "NC_001144.5"

# Start end index of the  of the PDC5 gene sequence
start_pdc5 = 410723
end_pdc5 = 412414



# Downlaoding PDC5 gene without reverse complement

seq_record_pdc5 = download_gene_sequence(accession_number_pdc5, start_pdc5, end_pdc5, 5)




# Accession number  PDC6 gene sequence
accession_number_pdc6 = "NC_001139.9"

# Start end index of the  of the PDC6 gene sequence
start_pdc6 = 651290
end_pdc6 = 652981



# Downlaoding PDC6 gene without reverse complement

seq_record_pdc6 = download_gene_sequence(accession_number_pdc6, start_pdc6, end_pdc6,6, True)





PDC  1 sequence download started
PDC  1 downloaded successfully

--------------------------------------------------

PDC  5 sequence download started
PDC  5 downloaded successfully

--------------------------------------------------

PDC  6 sequence download started
PDC  6 downloaded successfully

--------------------------------------------------



Reading the Fasta file and extracting the sequence from it

In [3]:
def extractSeq(file_name):
    for seq_record in list(SeqIO.parse(file_name, "fasta"))[:5]:
        
        
        pdc_gene_seq = str(seq_record.seq)
        
        return  pdc_gene_seq


Performing Blast to find snps

In [4]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML

def perform_blast(query_sequence):
    # Perform BLAST search
    # result_handle = NCBIWWW.qblast("blastn", "refseq_genomic", query_sequence)
    result_handle = NCBIWWW.qblast("blastn", "nr", query_sequence, entrez_query = 'txid4932[Organism]')
    
    return result_handle

def parse_blast_results(result_handle):
    # Retrieve BLAST results
    blast_records = NCBIXML.parse(result_handle)
    snp_strings = set()
    for blast_record in blast_records:
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                # Print alignment details
                query_seq = str(hsp.query).replace('-', '')  # Remove gaps from query sequence
                sbjct_seq = str(hsp.sbjct).replace('-', '')  # Remove gaps from subject sequence
                
                snp = []
                print("****Alignment****")
                print("Sequence:", alignment.title)
                print("Length:", alignment.length)
                print("E-value:", hsp.expect)
                print(hsp.query[0:75] + "...")
                print(hsp.match[0:75] + "...")
                print(hsp.sbjct[0:75] + "...")
                
                print()
                for i, (query_base, sbjct_base) in enumerate(zip(query_seq, sbjct_seq)):
                    if query_base != sbjct_base:
                        snp_strings.add(hsp.query_start + i)
                        snp.append(hsp.query_start + i)
                print("snp in this alignment : ", snp)
                print("Number of snp: ", len(snp))
                
                query_seq1 = query_seq
                while len(query_seq1) % 3 != 0:
                    query_seq1 += 'N'
                sbjct_seq1 = sbjct_seq
                while len(sbjct_seq1) % 3 != 0:
                    sbjct_seq1 += 'N'
                    
                print("translated query -> " + Seq(query_seq1).translate()[:75] + "............")
                print("translated subject -> " + Seq(sbjct_seq1).translate()[:75] + "............")
                        
    print("Number of snps ->", len(snp_strings))
    print("Some of the snps ............ ->", list(snp_strings)[:15])

                        
                        
    print("Number of snps ->", len(snp_strings))
    print("Some of the snps ............ ->", list(snp_strings)[:15])
                
                    
                
    print()

for i in [1,5,6]:
    print("\n--------------------------------------------------\n")
    print("For PDC ", i)
    file_name = "PDC" + str(i)  + ".fasta" 
    
    # Step 1: Prepare your query sequence
    query_sequence = extractSeq(file_name)
    
    # Step 2: Perform BLAST search
    result_handle = perform_blast(query_sequence)
    
    # Step 3: Parse and print BLAST results
    parse_blast_results(result_handle)
    print("\n--------------------------------------------------\n")

    


--------------------------------------------------

For PDC  1
****Alignment****
Sequence: gi|1586061137|gb|CP036478.1| Saccharomyces cerevisiae strain ySR128 chromosome XII, complete sequence
Length: 1076801
E-value: 0.0
ATGTCTGAAATTACTTTGGGTAAATATTTGTTCGAAAGATTAAAGCAAGTCAACGTTAACACCGTTTTCGGTTTG...
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
ATGTCTGAAATTACTTTGGGTAAATATTTGTTCGAAAGATTAAAGCAAGTCAACGTTAACACCGTTTTCGGTTTG...

snp in this alignment :  []
Number of snp:  0
translated query -> MSEITLGKYLFERLKQVNVNTVFGLPGDFNLSLLDKIYEVEGMRWAGNANELNAAYAADGYARIKGMSCIITTFG............
translated subject -> MSEITLGKYLFERLKQVNVNTVFGLPGDFNLSLLDKIYEVEGMRWAGNANELNAAYAADGYARIKGMSCIITTFG............
****Alignment****
Sequence: gi|1586061137|gb|CP036478.1| Saccharomyces cerevisiae strain ySR128 chromosome XII, complete sequence
Length: 1076801
E-value: 0.0
ATGTCTGAAATTACTTTGGGTAAATATTTGTTCGAAAGATTAAAGCAAGTCAACGTTAACACCGTTTTCGGTTTG...
||||||||||| || || ||||||||||| || ||||

Taking only those alignment in snp where only only on base pair is changing using filter of % identity

In [ ]:
from Bio.Blast import NCBIWWW, NCBIXML

# Record start time


# Define your gene sequence
gene_sequence = "ATGTCTGAAATTACTTTGGGTAAATATTTGTTCGAAAGATTAAAGCAAGTCAACGTTAACACCGTTTTCGGTTTGCCAGGTGACTTCAACTTGTCCTTGTTGGACAAGATCTACGAAGTTGAAGGTATGAGATGGGCTGGTAACGCCAACGAATTGAACGCTGCTTACGCCGCTGATGGTTACGCTCGTATCAAGGGTATGTCTTGTATCATCACCACCTTCGGTGTCGGTGAATTGTCTGCTTTGAACGGTATTGCCGGTTCTTACGCTGAACACGTCGGTGTTTTGCACGTTGTTGGTGTCCCATCCATCTCTGCTCAAGCTAAGCAATTGTTGTTGCACCACACCTTGGGTAACGGTGACTTCACTGTTTTCCACAGAATGTCTGCCAACATTTCTGAAACCACTGCTATGATCACTGACATTGCTACCGCCCCAGCTGAAATTGACAGATGTATCAGAACCACTTACGTCACCCAAAGACCAGTCTACTTAGGTTTGCCAGCTAACTTGGTCGACTTGAACGTCCCAGCTAAGTTGTTGCAAACTCCAATTGACATGTCTTTGAAGCCAAACGATGCTGAATCCGAAAAGGAAGTCATTGACACCATCTTGGCTTTGGTCAAGGATGCTAAGAACCCAGTTATCTTGGCTGATGCTTGTTGTTCCAGACACGACGTCAAGGCTGAAACTAAGAAGTTGATTGACTTGACTCAATTCCCAGCTTTCGTCACCCCAATGGGTAAGGGTTCCATTGACGAACAACACCCAAGATACGGTGGTGTTTACGTCGGTACCTTGTCCAAGCCAGAAGTTAAGGAAGCCGTTGAATCTGCTGACTTGATTTTGTCTGTCGGTGCTTTGTTGTCTGATTTCAACACCGGTTCTTTCTCTTACTCTTACAAGACCAAGAACATTGTCGAATTCCACTCCGACCACATGAAGATCAGAAACGCCACTTTCCCAGGTGTCCAAATGAAATTCGTTTTGCAAAAGTTGTTGACCACTATTGCTGACGCCGCTAAGGGTTACAAGCCAGTTGCTGTCCCAGCTAGAACTCCAGCTAACGCTGCTGTCCCAGCTTCTACCCCATTGAAGCAAGAATGGATGTGGAACCAATTGGGTAACTTCTTGCAAGAAGGTGATGTTGTCATTGCTGAAACCGGTACCTCCGCTTTCGGTATCAACCAAACCACTTTCCCAAACAACACCTACGGTATCTCTCAAGTCTTATGGGGTTCCATTGGTTTCACCACTGGTGCTACCTTGGGTGCTGCTTTCGCTGCTGAAGAAATTGATCCAAAGAAGAGAGTTATCTTATTCATTGGTGACGGTTCTTTGCAATTGACTGTTCAAGAAATCTCCACCATGATCAGATGGGGCTTGAAGCCATACTTGTTCGTCTTGAACAACGATGGTTACACCATTGAAAAGTTGATTCACGGTCCAAAGGCTCAATACAACGAAATTCAAGGTTGGGACCACCTATCCTTGTTGCCAACTTTCGGTGCTAAGGACTATGAAACCCACAGAGTCGCTACCACCGGTGAATGGGACAAGTTGACCCAAGACAAGTCTTTCAACGACAACTCTAAGATCAGAATGATTGAAATCATGTTGCCAGTCTTCGATGCTCCACAAAACTTGGTTGAACAAGCTAAGTTGACTGCTGCTACCAACGCTAAGCAATAA"  # Replace "ATCG..." with your gene sequence



# Store SNP strings in a list
snp_strings = set()

# Perform BLAST search
result_handle = NCBIWWW.qblast("blastn", "nr", gene_sequence, entrez_query = 'txid4932[Organism]')

# Parse BLAST results
blast_records = NCBIXML.parse(result_handle)
for blast_record in blast_records:
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            # Extract alignment information
            query_seq = hsp.query
            sbjct_seq = hsp.sbjct
            
            percent_identity = (hsp.identities / len(query_seq)) * 100
            if hsp.identities == len(query_seq)-1:  # Adjust the threshold as needed
                print("Subject ID:", alignment.title)
                print("Percent Identity:", percent_identity)
                print("Sequence:", hsp.sbjct)
                for i, (query_base, sbjct_base) in enumerate(zip(query_seq, sbjct_seq)):
                    if query_base != sbjct_base:
                        # snp_strings.append(f"SNP position: {hsp.query_start + i}, Query base: {query_base}, Subject base: {sbjct_base}")
                        snp_strings.add(hsp.query_start + i)
                print()
print ("SNP in the above results",snp_strings)


Translate the neucleotide sequence using BioPython


In [ ]:
def translate_sequence(sequence):
    seq = Seq(sequence)
    protein_sequence = seq.translate()
    return str(protein_sequence)

sequ = extractSeq("PDC1.fasta")
sequStrain = extractSeq("strain.fasta")
print(translate_sequence(sequ))
print(translate_sequence(sequStrain))

MSEITLGKYLFERLKQVNVNTVFGLPGDFNLSLLDKIYEVEGMRWAGNANELNAAYAADGYARIKGMSCIITTFGVGELSALNGIAGSYAEHVGVLHVVGVPSISAQAKQLLLHHTLGNGDFTVFHRMSANISETTAMITDIATAPAEIDRCIRTTYVTQRPVYLGLPANLVDLNVPAKLLQTPIDMSLKPNDAESEKEVIDTILALVKDAKNPVILADACCSRHDVKAETKKLIDLTQFPAFVTPMGKGSIDEQHPRYGGVYVGTLSKPEVKEAVESADLILSVGALLSDFNTGSFSYSYKTKNIVEFHSDHMKIRNATFPGVQMKFVLQKLLTTIADAAKGYKPVAVPARTPANAAVPASTPLKQEWMWNQLGNFLQEGDVVIAETGTSAFGINQTTFPNNTYGISQVLWGSIGFTTGATLGAAFAAEEIDPKKRVILFIGDGSLQLTVQEISTMIRWGLKPYLFVLNNDGYTIEKLIHGPKAQYNEIQGWDHLSLLPTFGAKDYETHRVATTGEWDKLTQDKSFNDNSKIRMIEIMLPVFDAPQNLVEQAKLTAATNAKQ*
MSEITLGKYLFERLSQVNCNTVFGLPGDFNLSLLDKLYEVKGMRWAGNANELNAAYAADGYARIKGMSCIITTFGVGELSALNGIAGSYAEHVGVLHVVGVPSISSQAKQLLLHHTLGNGDFTVFHRMSANISETTAMITDIANAPAEIDRCIRTTYTTQRPVYLGLPANLVDLNVPAKLLETPIDLSLKPNDAEAEAEVVRTVVELIKDAKNPVILADACASRHDVKAETKKLMDLTQFPVYVTPMGKGAIDEQHPRYGGVYVGTLSRPEVKKAVESADLILSIGALLSDFNTGSFSYSYKTKNIVEFHSDHIKIRNATFPGVQMKFALQKLLDAIPEVVKDYKPVAVPARVPITKSTPANTPMKQEWMWNHLGNFLREGDIVIAETGTSAFGINQTTFPTDVYAIVQVLWGSIGFTVGALLGATMAAEELDPK

Check if the two neucleotide translated sequence are same or not and if they are not then return the index


In [ ]:
def snpInTranslatedSequence(file1, file2):
    seq1 = extractSeq(file1)
    seq2 = extractSeq(file2)
    # print(seq1[255], seq2[255])
    
    seq1 = translate_sequence(seq1)
    seq2 = translate_sequence(seq2)
    print(seq1[255])
    print(seq1[255])
    print(seq2)
    if (len(seq1) != len(seq2)):
        return -1
    for i in range (len(seq1)):
        if (seq1[i] != seq2[i] and seq1[i] != 'X' and seq2[i] != 'X'):
            return i+1
    return -1

# file1 = "PDC_Gene5.fasta"
# file2 = "strain.fasta"
# print(snpInTranslatedSequence(file1, file2))
        
    